In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

from setup.univ_setup import *
from setup.utils import *
from factor_mining.combine_factors import *
from factor_mining.factor_preprocessing import *
from performance_analysis.pa_core import *

from backtest.BackTest import *
from backtest.regression_portfolio import *

from factor_mining.Mark0 import * # This is alpha file

In [2]:
%%time
### universe setup ###
datadir = '/home/derek-qi/Documents/R3000_Data/data/r3000/'
univ = univ_setup(datadir, version=5, wFund=True)
filt_na(univ)

# import warnings
# warnings.filterwarnings("ignore")

Setup R3000 universe
use existing binary file
CPU times: user 2.71 s, sys: 260 ms, total: 2.97 s
Wall time: 3.31 s


In [3]:
%%time
### model configuration ###
print('Calculate factors')
# define and calculate all factors
factors = alpha_wFund_00(univ)

factor_names = list(factors.keys())
factors_dict = factors
factor_names = [k for k in factors_dict.keys()]
N_f = len(factor_names)
datelst = sorted(factors_dict[factor_names[0]].keys())

# concat into factor exposure matrices
factor_exp_mat = combine_factors(factors)
filt_na(factor_exp_mat)

Calculate factors
CPU times: user 53.6 s, sys: 860 ms, total: 54.4 s
Wall time: 54.4 s


In [4]:
### Filt the universe ###
print('Filt the untradables')

filt_byval(univ, 'in_r3000', 0)
filt_byval(univ, 'price', 10)
# filt_byval(univ, 'not_in_MA', 0)
filt_byval(univ, 'volume', 300000)
filt_byval(univ, 'div_ratio', 10, keep='below')

factor_exp_mat = merge(factor_exp_mat, univ, right_cols=['f_log_ret_1'])
print('Done')

# Preprocessing factor expsure matrices
print('Standard scale each factor')
factor_exp_mat = process_batch(factor_exp_mat, standardize, factor_names=factor_names)
print('Winsorize with +/- 3 std')
factor_exp_mat = process_batch(factor_exp_mat, winsorize_std, factor_names=factor_names)
print('Done')

Filt the untradables
Done
Standard scale each factor
Winsorize with +/- 3 std
Done


In [ ]:
%%time
start_date = np.datetime64('2014-01-01')
end_date = np.datetime64('2016-12-31')

factor_names = list(factors.keys())
spcalc = RegressionPtflSpcalc(all_factor_names=factor_names, smoothing='simple')
bt = BackTest(univ=univ, factor_exp_mat=factor_exp_mat, daterange=[start_date, end_date], sp_calc=spcalc, rebal=1)
ptfl_lst, pnl_lst = bt.calc_pnl(silent=True, has_short=True)

In [ ]:
fr_series = np.stack(fr['factor_returns'] for fr in spcalc.all_factor_returns.values())
window = 30
ck = np.ones((window,)) / window
for c in range(3):
    fr_series[:,c] = np.convolve(ck, fr_series[:,c])[window-1:]
    plt.plot(fr_series[:,c], label=spcalc.all_factor_names[c])
plt.legend(loc='center right', bbox_to_anchor=(1.5, 0.5))
plt.show()

In [ ]:
bt.calc_pa(silent=False)

In [ ]:
dlst = sorted(univ.keys())
dlst = [d for d in dlst if d >= start_date and d <= end_date]
plt.plot(dlst, np.cumsum(pnl_lst['pnl']))
plt.show()

In [ ]:
tstamp = str(datetime.now())
bt.pnl_lst.to_csv('output/pnl_%s.csv' % tstamp)

In [ ]:
ptfl_lst = pd.concat(bt.ptfl_lst)
pp = pd.pivot_table(ptfl_lst, values='weight', columns='ticker', index='date')
pp.to_csv('output/portfolio_table_%s.csv' % tstamp)